In [1]:
# Importar librerías para tratamiento de datos
# -----------------------------------------------------------------------
import pandas as pd  # Pandas es una poderosa librería para manipulación y análisis de datos en Python.

# Importar librerías para web scraping y manipulación de datos
# -----------------------------------------------------------------------
from bs4 import BeautifulSoup
import requests

# Importar librerías para automatización de navegadores web con Selenium
# -----------------------------------------------------------------------
from selenium import webdriver  # Selenium es una herramienta para automatizar la interacción con navegadores web.
from webdriver_manager.chrome import ChromeDriverManager  # ChromeDriverManager gestiona la instalación del controlador de Chrome.
from selenium.webdriver.common.keys import Keys  # Keys es útil para simular eventos de teclado en Selenium.
from selenium.webdriver.support.ui import Select  # Select se utiliza para interactuar con elementos <select> en páginas web.

# Importar librerías para pausar la ejecución
# -----------------------------------------------------------------------
from time import sleep  # Sleep se utiliza para pausar la ejecución del programa por un número de segundos.

# Configuraciones
# -----------------------------------------------------------------------
pd.set_option('display.max_columns', None)  # Establece una opción de Pandas para mostrar todas las columnas de un DataFrame.

import numpy as np

import re
import random

In [2]:
# PASAMOS LAS LISTAS SACADAS DE LA API A DATAFRAME
df_action = pd.read_csv("action.csv", index_col = 0)
df_comedy = pd.read_csv("comedy.csv", index_col = 0)
df_drama = pd.read_csv("drama_no_duplicado.csv", index_col = 0)

In [3]:
#PASAMOS EL DATAFRAME QUE QUEREMOS USAR A LISTA Y COGEMOS LA CANTIDAD DE ELEMENTOS QUE QUERAMOS
primeras_drama_8000_df = df_drama.values.tolist()[0:8000]

In [5]:
def datos_imdb (lista):
    # Inicializar navegador

    driver = webdriver.Chrome() 

    # URL IMbd para extraer la información solicitada. 

    url = "https://www.imdb.com/" 

    # Hacer una solicitud GET a la URL especificada.

    driver.get(url)  

    # Maximizar la ventana - ¡ojo! con las posiciones de los productos según se abra el navegador = separar inspeccion con dock side en los 3 puntitos. 

    driver.maximize_window()

    sleep(5)

    # Aceptar cookies

    driver.find_element("css selector", "#__next > div > div > div.sc-jrcTuL.bPmWiM > div > button.icb-btn.sc-bcXHqe.sc-dkrFOg.sc-iBYQkv.dcvrLS.ddtuHe.dRCGjd").click() # Aceptar cookies

    # Creamos lista vacía para introducir el resultado de todas las peliculas con todos sus datos.

    total_pelis = [] #DURANTE LA EXTRACCION SACAMOS ESTA LISTA FUERA DE LA FUNCION POR SI DABA EL ERROR DEVTOOL (ERROR DEL NAVEGADOR) QUE SE NOS GUARDARA HASTA DONDE HUBIERA LLEGADO ANTES DEL ERROR Y NO PERDERLO TODO

    # Iniciamos bucle for por cada código accediendo por su indice.
    
    for indice, peli in enumerate(lista): #HACEMOS ENUMERATE PARA SABER CUANTAS PELIS LLEVAMOS
        try:
            sleep(3)
            driver.find_element("css selector","#suggestion-search").send_keys(peli[4], Keys.ENTER)  # Se introduce cada código en el campo de búsqueda y accede a la busqueda de la info que le vamos a pedir. 

        except: # SI NO ENCUENTRA LA BARRA DE BUSQUEDA PRINTAMOS ERROR 404, CLICKAMOS EN VOLVER A LA PAGINA DE INICIO Y VOLVEMOS A BUSCAR EL CODIGO
            print("404")
            driver.find_element("css selector","#error > div.error_message > a").click()
            sleep(3)

            driver.find_element("css selector","#suggestion-search").send_keys(peli[4], Keys.ENTER)
           
        pelicula = [] #Lista vacía para ir introduciendo todos los datos de cada pelicula
        
        #Puntuación

        try:

            puntuacion = driver.find_element("css selector","#__next > main > div > section.ipc-page-background.ipc-page-background--base.sc-304f99f6-0.fSJiHR > section > div:nth-child(4) > section > section > div.sc-491663c0-3.bdjVSf > div.sc-3a4309f8-0.bjXIAP.sc-69e49b85-1.llNLpA > div > div:nth-child(1) > a > span > div > div.sc-bde20123-0.dLwiNw > div.sc-bde20123-2.cdQqzc > span.sc-bde20123-1.cMEQkK").text.replace(",",".")
            
            pelicula.append(puntuacion)
        except: #EN EL CASO DE PUNTUACION TRAS COMPROBAR VARIAS VIMOS QUE SI NO CONSEGUIA LA PUNTUACION ERA PORQUE NO ESTABA, ASI QUE NO NECESITABAMOS OTRO SELECTOR
            
            pelicula.append("No tiene datos")

        # Ejecutamos Try/Except para obtener los datos de dirección, argumento, guionista y nombre de la película de la misma forma.
        # Se indica en el try/except varios selectores diferentes según localizados en la web en donde se pueda localizar cada dato. 
        # Con el except itera en varias opciones en el caso de que el primer try no haya funcionado , ejecutara el que encuentre, y sino imprime el mensaje de no hay datos. 

        # Dirección

        try:
            direccion = driver.find_element("css selector", "#__next > main > div > section.ipc-page-background.ipc-page-background--base.sc-304f99f6-0.fSJiHR > section > div:nth-child(4) > section > section > div.sc-491663c0-4.yjUiO > div.sc-491663c0-6.lnlBxO > div.sc-491663c0-10.rbXFE > section > div.sc-69e49b85-3.dIOekc > div > ul > li:nth-child(1) > div > ul > li > a").text
            
            pelicula.append(direccion)

        except:

            try:
                direccion = driver.find_element("css selector", "#__next > main > div > section.ipc-page-background.ipc-page-background--base.sc-304f99f6-0.fSJiHR > section > div:nth-child(4) > section > section > div.sc-491663c0-4.gEsAEH > div.sc-491663c0-6.eQRCDK > div.sc-491663c0-10.emoxHI > section > div.sc-69e49b85-3.dIOekc > div > ul > li:nth-child(1) > div > ul > li > a").text
                
                pelicula.append(direccion)

            except:

                try:
                    direccion = driver.find_element("css selector", "#__next > main > div > section.ipc-page-background.ipc-page-background--base.sc-304f99f6-0.fSJiHR > section > div:nth-child(4) > section > section > div.sc-491663c0-4.fVGakt > div > div.sc-491663c0-10.RRTJr > section > div.sc-69e49b85-3.dIOekc > div > ul > li:nth-child(1) > div > ul > li > a").text
                    
                    pelicula.append(direccion)

                except:
                    
                    pelicula.append("No tiene datos")

            
        # Guionista

        try:
            guionista = driver.find_element("css selector", "#__next > main > div > section.ipc-page-background.ipc-page-background--base.sc-304f99f6-0.fSJiHR > section > div:nth-child(4) > section > section > div.sc-491663c0-4.yjUiO > div.sc-491663c0-6.lnlBxO > div.sc-491663c0-10.rbXFE > section > div.sc-69e49b85-3.dIOekc > div > ul > li:nth-child(2) > div > ul > li > a").text
            #Usamos regex para separar los nombres de los guionistas
            patron = re.compile(r'([a-z])([A-Z])')
            # Usamos re.sub() para insertar una coma entre el nombre en minúsculas y la letra mayúscula
            resultado = re.sub(patron, r'\1, \2', guionista)
            
            pelicula.append(resultado)

        except:

            try:
                guionista = driver.find_element("css selector", "#__next > main > div > section.ipc-page-background.ipc-page-background--base.sc-304f99f6-0.fSJiHR > section > div:nth-child(4) > section > section > div.sc-491663c0-4.gEsAEH > div.sc-491663c0-6.eQRCDK > div.sc-491663c0-10.emoxHI > section > div.sc-69e49b85-3.dIOekc > div > ul > li:nth-child(2) > div > ul").text
                patron = re.compile(r'([a-z])([A-Z])')
                # Usamos re.sub() para insertar una coma entre el nombre en minúsculas y la letra mayúscula
                resultado = re.sub(patron, r'\1, \2', guionista)
                
                pelicula.append(resultado)

            except:

                try:
                    guionista = driver.find_element("css selector", "#__next > main > div > section.ipc-page-background.ipc-page-background--base.sc-304f99f6-0.fSJiHR > section > div:nth-child(4) > section > section > div.sc-491663c0-4.fVGakt > div > div.sc-491663c0-10.RRTJr > section > div.sc-69e49b85-3.dIOekc > div > ul > li:nth-child(2) > div").text
                    
                    pelicula.append(guionista)
                except:
                    
                    pelicula.append("No tiene datos")

        # Argumento
            
        try:
            
            argumento = driver.find_element("css selector", "#__next > main > div > section.ipc-page-background.ipc-page-background--base.sc-304f99f6-0.fSJiHR > section > div:nth-child(4) > section > section > div.sc-491663c0-4.yjUiO > div.sc-491663c0-6.lnlBxO > div.sc-491663c0-10.rbXFE > section > p > span.sc-466bb6c-2.chnFO").text
            
            if argumento != "":

                pelicula.append(argumento)

            elif argumento == "":
                pelicula.append("No tiene argumento")
                
        except:
            
            try:
                argumento = driver.find_element("css selector", "#__next > main > div > section.ipc-page-background.ipc-page-background--base.sc-304f99f6-0.fSJiHR > section > div:nth-child(4) > section > section > div.sc-491663c0-4.gEsAEH > div.sc-491663c0-6.eQRCDK > div.sc-491663c0-10.emoxHI > section > p > span.sc-466bb6c-2.chnFO").text
                
                if argumento != "":

                    pelicula.append(argumento)

                elif argumento == "":
                    pelicula.append("No tiene argumento") 

            except:
                
                pelicula.append("No tiene datos")


        # Duración (min)
        # 3 opciones de selector; 

        try:  
            try: # Primer selector, mayoría. 
            
                duracion_pre = driver.find_element("css selector", "#__next > main > div > section.ipc-page-background.ipc-page-background--base.sc-304f99f6-0.fSJiHR > section > div:nth-child(4) > section > section > div.sc-491663c0-3.bdjVSf > div.sc-69e49b85-0.jqlHBQ > ul > li:nth-child(3)").text
                duracion_str = duracion_pre.replace("h","").replace("min","").split() # Reemplazamos la h por vacio , y los minutos por espacio en blanco/vacio. 
                #print (duracion_str)  

                if len(duracion_str) == 2:                              # Primera condición ; si son 2 números, (hora y minutos) pasamos todo a minutos. 

                    duracion = int (duracion_str[0])*60 + int (duracion_str[1])
                    
                    pelicula.append(duracion)

                elif 'R' == duracion_str or "Not Rated" == duracion_str or "pg-13" == duracion_str or "g" == duracion_str or "pg" == duracion_str or "nc-17": # Segunda condición ; el anterior selector se utiliza para clasificación y la duración en otra parte. Seleccionamos nuevo selector a buscar.

                    duracion_pre = driver.find_element("css selector",'#__next > main > div > section.ipc-page-background.ipc-page-background--base.sc-304f99f6-0.fSJiHR > section > div:nth-child(4) > section > section > div.sc-491663c0-3.bdjVSf > div.sc-69e49b85-0.jqlHBQ > ul > li:nth-child(3)').text
                    duracion_str = duracion_pre.replace("h","").replace("min","").split()
                    #print("tiene R")
                    
                
                    if len(duracion_str) == 2:                                  
                        duracion = int(duracion_str[0])*60 + int (duracion_str[1])
                        
                        pelicula.append(duracion)

                    elif len(duracion_str) == 1 and "h" in duracion_pre:
                        duracion = int(duracion_str[0])*60
                        
                        pelicula.append(duracion)

                    else:
                        duracion = int (duracion_str[0])
                        
                        pelicula.append(duracion)            


                elif len(duracion_str) == 1 and "h" in duracion_pre:  # Si solo tiene un número y una h , lo pasamos a min. 
                        duracion = int(duracion_str[0])*60
                        
                        pelicula.append(duracion)

                else:                                                  # Directamente encontramos minutos.
                    duracion = int (duracion_str[0])
                    
                    pelicula.append(duracion)


            except:                                         # Tercer selector, por si las dos opciones primeras no funcionan. 

                duracion_pre = driver.find_element("css selector", '#__next > main > div > section.ipc-page-background.ipc-page-background--base.sc-304f99f6-0.fSJiHR > section > div:nth-child(4) > section > section > div.sc-491663c0-3.bdjVSf > div.sc-69e49b85-0.jqlHBQ > ul > li:nth-child(2)').text
                duracion_str = duracion_pre.replace("h","").replace("min","").split()
                
                if len(duracion_str) == 2:                                
                    duracion = int (duracion_str[0])*60 + int (duracion_str[1])
                    
                    pelicula.append(duracion)

                elif len(duracion_str) == 1 and "h" in duracion_pre:
                    duracion = int(duracion_str[0])*60
                    
                    pelicula.append(duracion)
                else:
                    duracion = int (duracion_str[0])
                    
                    pelicula.append(duracion)

        # Si no se cumplen ninguno de las condiciones anteriores, ejecuta no tiene datos. 
                    
        except:
            
            pelicula.append("No tiene datos")
            
        # Nombre de la película

        try:
            nombre_pelicula = driver.find_element("css selector", "#__next > main > div > section.ipc-page-background.ipc-page-background--base.sc-304f99f6-0.fSJiHR > section > div:nth-child(4) > section > section > div.sc-491663c0-3.bdjVSf > div.sc-69e49b85-0.jqlHBQ > h1 > span").text
            
            pelicula.append(nombre_pelicula)

        except:
            
            pelicula.append("No tiene datos")

        # Almacenamos en una variable , la conversión de la lista a tupla. 
            
        pelicula_tupla =  tuple(pelicula)
        total_pelis.append(pelicula_tupla)
        print(indice, pelicula_tupla)
        # Salimos a la pag.principal del navegador
        
        driver.back() 

    return total_pelis    # Impresion lista total 

In [ ]:
#Llamamos a la funcion
primeras_8000_drama_imdb = datos_imdb(primeras_drama_8000_df)

In [10]:
#convertimos la lista a dataframe
df_drama_imdb_resto = pd.DataFrame(primeras_8000_drama_imdb)

df_drama_imdb_resto

,0,1,2,3,4,5
0,5.8,Taylor Hackford,Art Linson,Un vistazo a la vida de un malhablado y enveje...,120,The Comedian
1,5.7,Warren Beatty,Warren Beatty,La historia de un amor poco convencional entre...,127,La excepción a la regla
2,No tiene datos,Marko Sanginetto,Brian Carlton,"To escape his memories, a man disappears into ...",No tiene datos,Mud Man
3,7.2,Duane Andersen,Duane Andersen,A superhero loses his powers shortly after tur...,80,Superpowerless
4,6.2,Susanna White,John le Carré,Una pareja se ve arrastrada a los planes de de...,108,Un traidor como los nuestros
...,...,...,...,...,...,...
4503,6.2,Jane Cusumano,Jane Cusumano,Despite different backgrounds and disapproving...,105,What Matters Most
4504,5.8,Dani Minnick,Dani Minnick,Set in California the story traces the intense...,94,Falling Like This
4505,6.0,Stéphane Giusti,"Raphaëlle Desplechin, Stéphane Giusti","Nella and Orfeo, communists, flee Tuscany with...",103,Bella ciao
4506,6.8,Deborah Attoinese,Deborah Attoinese,Three runaway girls hijack a car.,90,Zoe


In [11]:
#convertimos el dataframe a csv
df_drama_imdb_resto.to_csv("drama_imdb_resto_pelis.csv")